In [ ]:
# Librairie utilisées pour requêter des pages Web
import requests  
# Librairie utilisée pour le scrapping de pages web
from bs4 import BeautifulSoup
# Librairie utilisée pour la manipulation de données sous forme de dataframes
import pandas as pd

In [ ]:
def get_lyrics(artist, song_title):
    '''
    La fonction lyrique prend en entrée un artiste et un titre de chanson
    Cela nous permet d'obtenir l'URL de la page genius sur laquelle sont renseignées les paroles
    En cas d'échec lors du requêtage, un texte "Lyrics not found." sera retourné
    
    /!\ L'url ne pouvant pas dépasser une certaine taille, une condition if est présente en début de fonction
    /!\ La fonction actuelle ne prend pas en compte les featurings qui nécessitent une manipulation différente de 
    la chaine de caractère 
    Le volume de donnée récolté étant suffisament, la fonctionnalité n'a pas été developpée
    '''
    
    if len(artist) < 30 and len(song_title) < 30 : 
        # L'URL Genius
        url = f"https://genius.com/{artist.replace(' ', '-').lower()}-{song_title.replace(' ', '-').lower()}-lyrics"

        # Envoie de la requête
        response = requests.get(url)

        # Utilisation de BeautifulSoup pour récupérer le texte parmi les balises html
        soup = BeautifulSoup(response.text, 'html.parser')

        # Classe des divs contenant les lyrics
        lyrics_elements = soup.find_all("div", class_="Lyrics__Container-sc-1ynbvzw-1 kUgSbL")

        lyrics = ""
        for element in lyrics_elements:
            lyrics += element.get_text(separator='\n') + '\n\n'

        # Check if lyrics were found
        if lyrics.strip():
            return lyrics.strip()
        else:
            return "Lyrics not found."
    else:
            return "Lyrics not found."   

In [ ]:
data_musique = pd.read_csv("../Data/songs.csv", sep=';')

count_found = 0

# On stocke les chansons dans une liste
lyrics_list = []
for i in range(len(data_musique)) :
    # On parcourt toutes les lignes du dataframe 
    artist = data_musique['artist'][i]
    song_title = data_musique['name'][i]
    # On appelle la fonction get_lyrics
    lyrics = get_lyrics(artist, song_title)
    if lyrics != "Lyrics not found.":
        count_found +=1
        print(count_found, '/', i)
    
    lyrics_list.append(lyrics)
    
# On ajoute les paroles au dataframe
data_musique['lyrics'] = lyrics_list

In [ ]:
data_musique.to_csv("../Data/resultat_scrap.csv", sep=";")